Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [9]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [10]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
funds = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [11]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [12]:
fundsSI =  funds[funds[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
# fundsSI

In [13]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA', 'P/L' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,P/L
1,EUCA4,16.01,52.317067,134.0,2.267774,4.43,27.46,3.62
2,ALLD3,7.67,25.053323,133.0,2.266405,1.64,17.01,4.67
3,SOMA3,5.95,18.293851,132.0,2.074597,2.01,7.40,2.96
4,BRSR6,12.90,36.628636,131.0,1.839429,2.37,25.16,5.45
5,BMGB4,3.86,10.669208,130.0,1.764044,0.68,7.44,5.63
6,BRAP3,17.86,47.479206,129.0,1.658410,4.66,21.50,3.83
7,JHSF3,4.49,11.858446,128.0,1.641079,0.83,7.53,5.40
8,BAZA3,96.90,246.972857,127.0,1.548739,23.52,115.26,4.12
9,BRAP4,18.89,47.479206,126.0,1.513457,4.66,21.50,4.05
10,GOAU4,10.57,26.214576,125.0,1.480092,1.68,18.18,6.29


In [14]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,EUCA4,16.01,52.317067,134.0,2.267774,4.43,27.46
2,ALLD3,7.67,25.053323,133.0,2.266405,1.64,17.01
3,SOMA3,5.95,18.293851,132.0,2.074597,2.01,7.40
4,BRSR6,12.90,36.628636,131.0,1.839429,2.37,25.16
5,BMGB4,3.86,10.669208,130.0,1.764044,0.68,7.44
6,BRAP3,17.86,47.479206,129.0,1.658410,4.66,21.50
7,JHSF3,4.49,11.858446,128.0,1.641079,0.83,7.53
8,BAZA3,96.90,246.972857,127.0,1.548739,23.52,115.26
10,GOAU4,10.57,26.214576,125.0,1.480092,1.68,18.18
12,TRIS3,4.70,10.797812,123.0,1.297407,0.69,7.51


outros criterios adicionais

In [15]:
df_limpo =  df_limpo[(fundsSI['P/L'] > 0)]
top20 = df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,EUCA4,16.01,52.317067,134.0,2.267774,4.43,27.46
2,ALLD3,7.67,25.053323,133.0,2.266405,1.64,17.01
3,SOMA3,5.95,18.293851,132.0,2.074597,2.01,7.40
4,BRSR6,12.90,36.628636,131.0,1.839429,2.37,25.16
5,BMGB4,3.86,10.669208,130.0,1.764044,0.68,7.44
6,BRAP3,17.86,47.479206,129.0,1.658410,4.66,21.50
7,JHSF3,4.49,11.858446,128.0,1.641079,0.83,7.53
8,BAZA3,96.90,246.972857,127.0,1.548739,23.52,115.26
10,GOAU4,10.57,26.214576,125.0,1.480092,1.68,18.18
12,TRIS3,4.70,10.797812,123.0,1.297407,0.69,7.51


In [16]:
df_limpo =  df_limpo[(fundsSI['DIVIDA LIQUIDA / EBIT'] < 3.1) ]
top20 = df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,EUCA4,16.01,52.317067,134.0,2.267774,4.43,27.46
2,ALLD3,7.67,25.053323,133.0,2.266405,1.64,17.01
3,SOMA3,5.95,18.293851,132.0,2.074597,2.01,7.40
6,BRAP3,17.86,47.479206,129.0,1.658410,4.66,21.50
7,JHSF3,4.49,11.858446,128.0,1.641079,0.83,7.53
10,GOAU4,10.57,26.214576,125.0,1.480092,1.68,18.18
14,SCAR3,23.46,52.991584,121.0,1.258806,4.28,29.16
16,CLSC4,79.90,179.305981,119.0,1.244130,16.43,86.97
18,SAPR3,5.59,12.314625,117.0,1.202974,1.00,6.74
21,DMVF3,7.10,14.425368,114.0,1.031742,0.53,17.45


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.